In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from collections import defaultdict
from surprise import Dataset, Reader, KNNWithMeans, accuracy
from surprise.model_selection import GridSearchCV
from surprise.model_selection import train_test_split
#from sklearn.model_selection import train_test_split as tts
from surprise.model_selection import KFold
from surprise import Reader, Dataset, SVD#, evaluate
from surprise import Dataset, SVD
from surprise.model_selection import cross_validate

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/recsys-in-practice/sample_submission.csv
/kaggle/input/recsys-in-practice/test_joke_df_nofactrating.csv
/kaggle/input/recsys-in-practice/train_joke_df.csv


In [5]:
reader = Reader(rating_scale=(-10, 10))

In [6]:
ratings = pd.read_csv('/kaggle/input/recsys-in-practice/train_joke_df.csv')
ratings.head()

,UID,JID,Rating
0,18029,6,-1.26
1,3298,64,-4.17
2,3366,58,0.92
3,12735,92,3.69
4,11365,38,-6.60


In [7]:
data = Dataset.load_from_df(ratings[['UID', 'JID', 'Rating']], reader)

In [11]:
param_grid_svd = {
    'n_factors': [160], 
    'n_epochs': [40], 
    'lr_all': [0.002], 
    'reg_all': [0.02]
    }

gs_svd = GridSearchCV(SVD, param_grid_svd, measures=['rmse', 'mae'], cv=5, joblib_verbose=20, n_jobs=4, refit=True)
gs_svd.fit(data)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:  1.1min
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:  1.1min remaining:  1.7min
[Parallel(n_jobs=4)]: Done   3 out of   5 | elapsed:  1.2min remaining:   46.6s
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:  1.7min remaining:    0.0s
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:  1.7min finished


In [16]:
gs_svd.best_score

{'rmse': 4.800420590126338, 'mae': 3.797703664920109}

In [12]:
test = pd.read_csv('/kaggle/input/recsys-in-practice/test_joke_df_nofactrating.csv', index_col=0)
test.head(5)

,UID,JID
InteractionID,,
0,11228,39
1,21724,85
2,16782,56
3,12105,42
4,14427,2


In [13]:
test['Rating'] = test[['UID', 'JID']].apply(lambda x: gs_svd.predict(x[0], x[1]).est,
                                                      axis = 1)

In [14]:
test['Rating'].to_frame().head(5)

,Rating
InteractionID,
0,1.55155
1,1.00000
2,1.00000
3,5.00000
4,5.00000


In [15]:
test['Rating'].to_frame().to_csv('submission.csv')